In [ ]:
import os
import psycopg
import numpy as np
from watcher.models import WatcherGui
from watcher.preprocess import preprocess_for_inference
from watcher.utils import get_gpu_devices, get_mig_devices, load_psycopg_params, shuffle_timeline_matrix_indexes
from watcher.general_params import watcher_config as config

### Step1: Checl your available GPU devices (UUID)

In [ ]:
mig_devices = get_mig_devices()
if mig_devices:
    print("MIG devices:")
    for d in mig_devices:
        print(d)
gpu_devices = get_gpu_devices()
if gpu_devices:
    print("GPU devices:")
    for d in gpu_devices:
        print(d)

# Device list will be desplayed below

### Step2: Set your GPU to use

In [ ]:
device = 'Set your gpu device here'

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = device

### Step3: Set model blueprint, and instantiate GUI class

In [ ]:
blueprint = "Specify the path to the model blueprint here"

In [ ]:
# Instantiate a gui model
gui = WatcherGui(blueprint=blueprint, 
                 temperature=1.0,
                 device="cuda:0",)

### Step4: Setup data base schema, and pick up a sample patient data

In [ ]:
# Specify schema name if it is not 'public'
schema = "public"

In [ ]:
# Set up patient ID, or set this to None for random sampling 
patient_id = None

# If patient ID is None, randomly selected.
if patient_id is None:
    query = f"SELECT patient_id FROM {schema}.patients ORDER BY RANDOM() LIMIT 1;"

    with psycopg.connect(load_psycopg_params()) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            result = cur.fetchone()
            patient_id = result[0] if result else None

print("Selected patient:", patient_id)

In [ ]:
input_matrix, catalog_indexes, dob = preprocess_for_inference(patient_id=patient_id,
                                                              db_schema=schema,
                                                              model=gui.model,
                                                              start=None,
                                                              end=None)
print(input_matrix.size())

### Step5: Run GUI demo.

In [ ]:
# Keep these codes as is
shuffle = config.SHUFFLE_INPUT
if shuffle:
    shuffled_idx = shuffle_timeline_matrix_indexes(timeline_matrix=input_matrix.squeeze(0),
                                                pad_start=None,
                                                dsc_idx=gui.model.dsc_index,
                                                eot_idx=gui.model.eot_index,
                                                lab_code_token_idx=gui.model.lab_token_index,
                                                k=1,
                                                max_random_integer=1000)[0]
    shuffled_matrix = input_matrix[:, shuffled_idx, :]
    shuffled_indexes = np.array(catalog_indexes)[shuffled_idx].tolist()
    gui.set_timeline(shuffled_matrix, shuffled_indexes)
else:
    gui.set_timeline(input_matrix, catalog_indexes)

In [ ]:
# Run the GUI demo
gui.demo_retrospective(dpi=500)
